# Hydromet


## Import data

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import ftplib
plt.rcParams["figure.figsize"] = (20,16)

In [22]:
    
def download_files(folder_local, folder_server):
    load_dotenv()
    with ftplib.FTP(os.getenv("FTP_SERVER"), os.getenv("FTP_USERNAME"), os.getenv("FTP_PASSWORD")) as ftp:
        ftp.cwd(folder_server)
        filenames = ftp.nlst()
  
        for filename in filenames:

            with open(f"../data/" + folder_local + "/" + filename, "ab") as file:
                # Command for Downloading the file "RETR filename"
                ftp.retrbinary('RETR ' + filename, file.write)


    
    
def get_values(path:str):
    
    df = pd.read_csv(path, sep=";" )
    df = df[df.columns[1::2]]
    df = pd.DataFrame.transpose(df)   
    df = df.dropna(axis=0, how='all')
    df = pd.DataFrame.transpose(df)   

    df.columns.values[0] = 'time_temp'
    df['time_temp'] = df['time_temp'].astype('int64')
    df['time_temp'] = df['time_temp'].astype(str)
    
    temp = pd.to_datetime(df['time_temp'], format='%y%m%d%H%M%S')
    df = df.loc[ : , df.columns != 'time_temp'].apply(pd.to_numeric, errors = 'coerce')

    df = df.iloc[:, :-1]
    
    df["datetime"] = temp
    
    return df

def outliers(df, feature):
    Q1= df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    upper_limit = Q3 + 1.5 * IQR
    lower_limit = Q1 - 1.5 * IQR
    return upper_limit, lower_limit

def make_dataframe(folder_local, folder_server, start_date):
    
    start=True
    df=None
    for filename in glob.glob(folder_local +"*.txt"):
        date=int(filename[filename.index("_", -20, -1)+1:filename.index("_", -15, -1)])
        if date>= start_date:
            if start:  
                df = get_values(filename) 
                start =False
            else:
                to_append = get_values(filename) 
                df = df.append(to_append, ignore_index=True)
    if df is  None:
        return None
    col_names = [  "AVGVi","AVGCi","OCi","Temp4","NTU4", "Turb4","Temp3","Conductivity", "A_samp", "Samples", "P1", "RWL", "geolux_TIS", "geolux_TW", "geolux_x", "geolux_y", "geolux_p_7", "datetime"]

    for i in range(len(col_names)):
        df.columns.values[i] = col_names[i]
        
    df = df.set_index('datetime')
    return df    


In [23]:
# Download files
folder_local = "../data/Hydromet/"
folder_server = "Hydromet"
start_date=220822
# Read all files
#download_files(folder_local, folder_server)
df = make_dataframe(folder_local, folder_server, start_date)
df



220824
0    2.208241e+11
1    2.208241e+11
2    2.208241e+11
3    2.208241e+11
4    2.208241e+11
5    2.208241e+11
Name: time_temp, dtype: float64
0    220824051021
1    220824052023
2    220824053021
3    220824054023
4    220824055021
5    220824060022
Name: time_temp, dtype: int64
220826
0    220826091522
1    220826093023
2    220826094523
3    220826100021
Name: time_temp, dtype: object
0    220826091522
1    220826093023
2    220826094523
3    220826100021
Name: time_temp, dtype: int64
220823
0    220823120100
1    220823121021
2    220823122021
3    220823123021
4    220823124023
5    220823125021
6    220823130023
Name: time_temp, dtype: object
0    220823120100
1    220823121021
2    220823122021
3    220823123021
4    220823124023
5    220823125021
6    220823130023
Name: time_temp, dtype: int64
220809
220804
220828
0    2.208280e+11
1    2.208280e+11
2    2.208280e+11
3    2.208281e+11
Name: time_temp, dtype: float64
0    220828041521
1    220828043021
2    220828044522
3   

: 

: 

In [ ]:
df
#df

## Tests

In [ ]:
test = get_values( "../data/Hydromet/YDOC_Hydromet_test_unit_01_110052556_220825_090031.txt")
test


In [ ]:
col_names = [  "AVGVi","AVGCi","OCi","Temp4","NTU4", "Turb4","Temp3","Conductivity", "A_samp", "Samples", "P1", "RWL", "geolux_TIS", "geolux_TW", "geolux_x", "geolux_y", "geolux_p_7", "datetime"]




test.columns = col_names
   
#df = df.set_index('datetime')
test

## Inspect values

In [ ]:
df

In [ ]:
fig, axs = plt.subplots(6, 1 , figsize=(20,16))

df = df[(df.index > '2022-08-22') ]


col_name = "geolux_TIS"
upper, lower = outliers(df, col_name)

filtered= df.loc[(df[col_name] > lower) & (df[col_name] < upper)].sort_values('datetime')
axs[0].plot(filtered.index, filtered[col_name])
axs[0].set_xlabel('time')
axs[0].set_ylabel('Temperature outside [°C]')


col_name = "Temp4"
upper, lower = outliers(df, col_name)

filtered= df.loc[(df[col_name] > lower) & (df[col_name] < upper)].sort_values('datetime')
axs[1].plot(filtered.index, filtered[col_name])
axs[1].set_xlabel('time')
axs[1].set_ylabel('Temperature water [°C]')


col_name = "Conductivity"
upper, lower = outliers(df, col_name)

filtered= df.loc[(df[col_name] > lower) & (df[col_name] < upper)].sort_values('datetime')
axs[2].plot(filtered.index, filtered[col_name])
axs[2].set_xlabel('time')
axs[2].set_ylabel('Conductivity [µs/cm]')


col_name = "AVGVi"
upper, lower = outliers(df, col_name)

filtered= df.loc[(df[col_name] > lower) & (df[col_name] < upper)].sort_values('datetime')
axs[3].plot(filtered.index, filtered[col_name])
axs[3].set_xlabel('time')
axs[3].set_ylabel('Voltage [V]')


col_name = "Turb4"
upper, lower = outliers(df, col_name)

filtered= df.loc[(df[col_name] > lower) & (df[col_name] < upper)].sort_values('datetime')
axs[4].plot(filtered.index, filtered[col_name])
axs[4].set_xlabel('time')
axs[4].set_ylabel('Turbidity [NTU]')


col_name = "RWL"
upper, lower = outliers(df, col_name)

filtered= df.loc[(df[col_name] > lower) & (df[col_name] < upper)].sort_values('datetime')
axs[5].plot(filtered.index, filtered[col_name])
axs[5].set_xlabel('time')
axs[5].set_ylabel('Relative Water level [mm]')




In [ ]:
fig2, ax = plt.subplots(1, 1 , figsize=(16,12))



col_name = "Temp4"
upper, lower = outliers(df, col_name)

filtered= df.loc[(df[col_name] > lower) & (df[col_name] < upper)].sort_values('Temp3')
ax.plot(filtered["Temp3"], filtered["Temp4"])
ax.set_xlabel('Temp3')
ax.set_ylabel('Temp4')
